# Pixel Intensity-Based Colocalization

<div class="custom-button-row">
    <a 
        class="custom-button custom-download-button" href="../../../notebooks/08_colocalization/pixel_intensity_based_colocalization.ipynb" download>
        <i class="fas fa-download"></i> Download this Notebook
    </a>
    <a
    class="custom-button custom-download-button" href="https://colab.research.google.com/github/HMS-IAC/bobiac/blob/gh-pages/colab_notebooks/08_colocalization/pixel_intensity_based_colocalization.ipynb" target="_blank">
        <img class="button-icon" src="../../../_static/logo/icon-google-colab.svg" alt="Open in Colab">
        Open in Colab
    </a>
</div>

In [ ]:
# /// script
# requires-python = ">=3.12"
# dependencies = [
#     "matplotlib",
#     "ndv[jupyter,vispy]",
#     "scikit-image",
#     "scipy",
#     "tifffile",
#     "imagecodecs",
# ]
# ///

<!-- <div align="center"> <img src="https://raw.githubusercontent.com/HMS-IAC/bobiac/main/_static/images/coloc/pearsons.png" alt="Ilastik Logo" width="600"></div> -->
<div align="left"> <img src="../../_static/images/coloc/pearsons.png" alt="Ilastik Logo" width="600"></div>

In [ ]:
# numerator = np.sum((image1 - mean_image1) * (image2 - mean_image2))
# denominator = np.sqrt(
#     np.sum((image1 - mean_image1) ** 2) * np.sum((image2 - mean_image2) ** 2)
# )
# prs = numerator / denominator

# pearson, p_value = pearsonr(image1.flatten(), image2.flatten())

<!-- <div align="center"> <img src="https://raw.githubusercontent.com/HMS-IAC/bobiac/main/_static/images/coloc/manders.png" alt="Ilastik Logo" width="600"></div> -->
<div align="left"> <img src="../../_static/images/coloc/manders.png" alt="Ilastik Logo" width="600"></div>

In [ ]:
# # create binary masks based on thresholds
# image_1_mask = image1 > thr1
# image_2_mask = image2 > thr2

# # only consider pixels within the mask (keep intensities above threshold, zero elsewhere)
# image1_tr = np.where(image_1_mask, image1, 0)
# image2_tr = np.where(image_2_mask, image2, 0)

# # find overlapping regions
# # use intensity values from image1 (above threshold) ONLY where image2 > 0
# image1_overlap_with_image2 = np.where(image_2_mask, image1_tr, 0)
# # use intensity values from image2 (above threshold) ONLY where image1 > 0
# image2_overlap_with_image1 = np.where(image_1_mask, image2_tr, 0)

# # calculate the sum of the masked images
# sum_image1_tr = np.sum(image1_tr)
# sum_image2_tr = np.sum(image2_tr)

# # calculate the Manders coefficients
# M1 = np.sum(image1_overlap_with_image2) / sum_image1_tr
# M2 = np.sum(image2_overlap_with_image1) / sum_image2_tr


# # def manders_colocalization_coefficients(
# #     image1: np.ndarray,
# #     image2: np.ndarray,
# #     image_1_mask: np.ndarray,
# #     image_2_mask: np.ndarray,
# # ) -> tuple[float, float]:
# #     """Calculate the Manders colocalization coefficients for two images.

# #     Parameters
# #     ----------
# #     image1 : np.ndarray
# #         The first image.
# #     image2 : np.ndarray
# #         The second image.
# #     image_1_mask : np.ndarray
# #         The mask for the first image.
# #     image_2_mask : np.ndarray
# #         The mask for the second image.

# #     Returns
# #     -------
# #     tuple[float, float]
# #         The Manders coefficients M1 and M2.
# #         M1 is the fraction of image1 that colocalizes with image2.
# #         M2 is the fraction of image2 that colocalizes with image1.
# #     """

# #     # only consider pixels within the mask
# #     image1_tr = np.where(image_1_mask, image1, 0)
# #     image2_tr = np.where(image_2_mask, image2, 0)

# #     # use intensity values from image1 where image2 > 0
# #     image1_coloc = np.where(image_2_mask, image1_tr, 0)
# #     # use intensity values from image2 where image1 > 0
# #     image2_coloc = np.where(image_1_mask, image2_tr, 0)

# #     sum_image1_tr = np.sum(image1_tr)
# #     sum_image2_tr = np.sum(image2_tr)

# #     M1 = np.sum(image1_coloc) / sum_image1_tr if sum_image1_tr > 0 else 0
# #     M2 = np.sum(image2_coloc) / sum_image2_tr if sum_image2_tr > 0 else 0

# #     return M1, M2